<a href="https://colab.research.google.com/github/sanghaimuskan/English-to-French/blob/master/Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
import nltk

In [98]:
nltk.download('stopwords')
nltk.download('twitter_samples')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [102]:
import re
import string

import numpy as np
import pandas as pd
from nltk.corpus import stopwords, twitter_samples
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [60]:
def cosine_similarity(A, B):
    
    # you have to set this variable to the true label.
    cos = -10
    dot = np.dot(A, B)
    norma = np.linalg.norm(A)
    normb = np.linalg.norm(B)
    cos = dot / (norma * normb)

    return cos


In [100]:
def process_tweet(tweet):
    
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [63]:
def get_dict(file_name):
    
    my_file = pd.read_csv(file_name, delimiter=' ')
    etof = {}  # the english to french dictionary to be returned
    for i in range(len(my_file)):
        # indexing into the rows.
        en = my_file.loc[i][0]
        fr = my_file.loc[i][1]
        etof[en] = fr

    return etof


# **The word embeddings data for English and French words**

The full dataset for English embeddings is about 3.64 gigabytes, and the French embeddings are about 629 megabytes. so we have taken the subset.


In [43]:
from google.colab import files
uploaded = files.upload()


Saving utf-8''en_embeddings.p to utf-8''en_embeddings.p
Saving utf-8''fr_embeddings.p to utf-8''fr_embeddings.p


In [46]:
import pickle
en_embeddings_subset = pickle.load(open("utf-8''en_embeddings.p", "rb"))
fr_embeddings_subset = pickle.load(open("utf-8''fr_embeddings.p", "rb"))

# **Loading two dictionaries one for training and other for testing**

In [68]:
uploaded = files.upload()

Saving en-fr.test.txt to en-fr.test.txt
Saving en-fr.train.txt to en-fr.train.txt


In [70]:
en_fr_train = get_dict('en-fr.train.txt')
print('The length of the English to French training dictionary is', len(en_fr_train))
en_fr_test = get_dict('en-fr.test.txt')
print('The length of the English to French test dictionary is', len(en_fr_train))

The length of the English to French training dictionary is 5000
The length of the English to French test dictionary is 5000


# **Generate embedding and transform matrices**

Translating English to French using embeddings


In [78]:
"""
    Input:
        en_fr: English to French dictionary
        french_vecs: French words to their corresponding word embeddings.
        english_vecs: English words to their corresponding word embeddings.
    Output: 
        X: a matrix where the columns are the English embeddings.
        Y: a matrix where the columns correspong to the French embeddings.
        R: the projection matrix that minimizes the F norm ||X R -Y||^2.
"""

def get_matrices(en_fr, french_vecs, english_vecs):
  # X_l and Y_l are lists of the english and french word embeddings
  X_l = list()
  Y_l = list()

  english_set = english_vecs.keys()
  french_set = french_vecs.keys()

  for en_word, fr_word in en_fr.items():
    if fr_word in french_set and en_word in english_set:

      en_vec = english_vecs[en_word]
      fr_vec = french_vecs[fr_word]

      X_l.append(en_vec)
      Y_l.append(fr_vec)

  X = np.vstack(X_l)
  Y = np.vstack(Y_l)

  return X, Y

In [79]:
X_train, Y_train = get_matrices(en_fr_train, fr_embeddings_subset, en_embeddings_subset)

## **Translations as linear transformation of embeddings**

We have to create R (transformation matrix)

Find a matrix R that minimizes the loss function



In [80]:
def compute_loss(X, Y, R):
  m = X.shape[0]
  diff = np.dot(X,R) - Y
  diff_squared = diff**2

  sum_diff = np.sum(diff_squared)
  diff_norm = sum_diff/m              #loss

  return diff_norm


## computing the gradient loss in respect to transform matrix R

In [81]:
def compute_gradient (X, Y, R):
  m = X.shape[0]

  grad = 2/m *np.dot(X.T, (np.dot(X,R)-Y))

  return grad

# **Finding an optimal R with gradient descent**

In [84]:
def align_embeddings(X, Y, iter = 100, learning_rate = 0.0003):
  np.random.seed(120)
  R = np.random.rand(X.shape[1],X.shape[1])

  for i in range(iter):
    gradient = compute_gradient(X, Y, R)

    R -= learning_rate*gradient
  
  return R

In [87]:
R_train = align_embeddings(X_train, Y_train, iter=400, learning_rate=0.8)

# **K-Nearest Neighbours Algorithm**

It takes a vector as input and finds the other vectors in the dataset that are closest to it

## **Cosine similarity**
Cosine similarity between vectors  𝑢  and  𝑣  calculated as the cosine of the angle between them.

Note: Distance and similarity are pretty much opposite things.
We can obtain distance metric from cosine similarity, but the cosine similarity can't be used directly as the distance metric.
When the cosine similarity increases (towards  1 ), the "distance" between the two vectors decreases (towards  0 ).
We can define the cosine distance between  𝑢  and  𝑣  as
𝑑cos(𝑢,𝑣)=1−cos(𝑢,𝑣)

In [88]:
def nearest_neighbor(v, candidates, k=1):
  similarity_l = []

  for row in candidates:
    cos_similarity = cosine_similarity(v,row)
    similarity_l.append(cos_similarity)

  sorted_ids = np.argsort(similarity_l)
  k_idx = sorted_ids[-k:]

  return k_idx

In [94]:
def test(X, Y, R):
  pred = np.dot(X,R)

  num_correct = 0

  for i in range(len(pred)):
    pred_id = nearest_neighbor(pred[i], Y)

    if pred_id == i:
      num_correct +=1
  
  accuracy = num_correct / len(pred)
  return accuracy

In [91]:
X_val, Y_val = get_matrices(en_fr_test, fr_embeddings_subset, en_embeddings_subset)

In [95]:
acc = test(X_val, Y_val, R_train)  # this might take a minute or two
print(f"accuracy on test set is {acc:.3f}")

accuracy on test set is 0.550


# **LSH and Document search**
More efficient version of k-nearest neighbors using locality sensitive hashing

In [103]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

# **1. Getting the document embeddings**

## **Bag of Words document model**

Text document sequence
example: apple is better than orange and orange is better than apple have different meaning


In [104]:
def document_embedding(tweet, en_embeddings):
  doc_embedding = np.zeros(300)
  processed_doc = process_tweet(tweet)

  for word in processed_doc:
    doc_embedding+=en_embeddings.get(word,0)
  return doc_embedding

In [105]:
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"
tweet_embedding = document_embedding(custom_tweet, en_embeddings_subset)
tweet_embedding[-5:]

array([-0.00268555, -0.15378189, -0.55761719, -0.07216644, -0.32263184])

In [107]:
def get_document_vecs(all_docs, en_embeddings):
  ind2Doc_dict = {}
  document_vec_l = []

  for i, doc in enumerate(all_docs):
    doc_embedding = document_embedding(doc, en_embeddings)

    ind2Doc_dict[i] = doc_embedding

    document_vec_l.append(doc_embedding)
  # convert the list of document vectors into a 2D array (each row is a document vector)
  document_vec_matrix = np.vstack(document_vec_l)

  return document_vec_matrix, ind2Doc_dict

In [109]:
all_tweets = all_negative_tweets + all_positive_tweets
document_vecs, ind2Tweet = get_document_vecs(all_tweets, en_embeddings_subset)

Each plane divides the space to $2$ parts.
So $n$ planes divide the space into $2^{n}$ hash buckets.
We want to organize 10,000 document vectors into buckets so that every bucket has about $~16$ vectors.
For that we need $\frac{10000}{16}=625$ buckets.
We're interested in $n$, number of planes, so that $2^{n}= 625$. Now, we can calculate $n=\log_{2}625 = 9.29 \approx 10$.

In [111]:
# The number of planes. We use log2(625) to have ~16 vectors/bucket.
N_PLANES = 10
# Number of times to repeat the hashing to improve the search.
N_UNIVERSES = 25

In [114]:
N_VECS = len(all_tweets)       # This many vectors.
N_DIMS = len(ind2Tweet[1])     # Vector dimensionality.

# **Getting a hash number for a vector**

### **Hyperlanes in vector space**
In $3$-dimensional vector space, the hyperplane is a regular plane. In $2$ dimensional vector space, the hyperplane is a line.
Generally, the hyperplane is subspace which has dimension $1$ lower than the original vector space has.
A hyperplane is uniquely defined by its normal vector.
Normal vector $n$ of the plane $\pi$ is the vector to which all vectors in the plane $\pi$ are orthogonal (perpendicular in $3$ dimensional case)


In [115]:
np.random.seed(0)
planes_l = [np.random.normal(size=(N_DIMS, N_PLANES))for _ in range(N_UNIVERSES)]

In [121]:
def hash_value_of_vector(v, planes):
  dot_product = np.dot(v, planes)
  sign_of_dot = np.sign(dot_product)
  h = sign_of_dot>=0

  # remove extra un-used dimensions (convert this from a 2D to a 1D array)
  h = np.squeeze(h)

  hash_value = 0
  n_planes = planes.shape[1]

  for i in range (n_planes):
    hash_value += 2**i*h[i]
  # cast hash_value as an integer
  hash_value = int(hash_value)

  return hash_value

In [122]:
np.random.seed(0)
idx = 0
planes = planes_l[idx]  # get one 'universe' of planes to test the function
vec = np.random.rand(1, 300)
print(f" The hash value for this vector,",f"and the set of planes at index {idx},",f"is {hash_value_of_vector(vec, planes)}")

 The hash value for this vector, and the set of planes at index 0, is 768


# **Making Hash Table**


In [125]:
def make_hash_table(vecs,planes):
  num_of_planes = planes.shape[1]
  num_buckets = 2**num_of_planes

  # create the hash table as a dictionary.
  # Keys are integers (0,1,2.. number of buckets)
  # Values are empty lists
  hash_table = {i:[] for i in range(num_buckets)}
  # create the id table as a dictionary.
  # Keys are integers (0,1,2... number of buckets)
  # Values are empty lists
  id_table = {i:[] for i in range(num_buckets)}

  for i, v in enumerate(vecs):

        # calculate the hash value for the vector
        h = hash_value_of_vector(v,planes)
        #print(h)
        #print('******')
        # store the vector into hash_table at key h,
        # by appending the vector v to the list at key h
        hash_table[h].append(v)

        # store the vector's index 'i' (each document is given a unique integer 0,1,2...)
        # the key is the h, and the 'i' is appended to the list at key h
        id_table[h].append(i)
  return hash_table, id_table

In [126]:
np.random.seed(0)
planes = planes_l[0]  # get one 'universe' of planes to test the function
vec = np.random.rand(1, 300)
tmp_hash_table, tmp_id_table = make_hash_table(document_vecs, planes)

print(f"The hash table at key 0 has {len(tmp_hash_table[0])} document vectors")
print(f"The id table at key 0 has {len(tmp_id_table[0])}")
print(f"The first 5 document indices stored at key 0 of are {tmp_id_table[0][0:5]}")

The hash table at key 0 has 3 document vectors
The id table at key 0 has 3
The first 5 document indices stored at key 0 of are [8276, 8281, 8282]


In [127]:
# Creating the hashtables
hash_tables = []
id_tables = []
for universe_id in range(N_UNIVERSES):  # there are 25 hashes
    print('working on hash universe #:', universe_id)
    planes = planes_l[universe_id]
    hash_table, id_table = make_hash_table(document_vecs, planes)
    hash_tables.append(hash_table)
    id_tables.append(id_table)

working on hash universe #: 0
working on hash universe #: 1
working on hash universe #: 2
working on hash universe #: 3
working on hash universe #: 4
working on hash universe #: 5
working on hash universe #: 6
working on hash universe #: 7
working on hash universe #: 8
working on hash universe #: 9
working on hash universe #: 10
working on hash universe #: 11
working on hash universe #: 12
working on hash universe #: 13
working on hash universe #: 14
working on hash universe #: 15
working on hash universe #: 16
working on hash universe #: 17
working on hash universe #: 18
working on hash universe #: 19
working on hash universe #: 20
working on hash universe #: 21
working on hash universe #: 22
working on hash universe #: 23
working on hash universe #: 24


# **Approximate K-NN**

In [134]:
def aprx_knn(doc_id, v, planes_l, k=1, num_universes_to_use = N_UNIVERSES):
    
    

    # Vectors that will be checked as possible nearest neighbor
    vecs_to_consider_l = list()

    # list of document IDs
    ids_to_consider_l = list()

    # create a set for ids to consider, for faster checking if a document ID already exists in the set
    ids_to_consider_set = set()

    # loop through the universes of planes
    for universe_id in range(num_universes_to_use):

        # get the set of planes from the planes_l list, for this particular universe_id
        planes = planes_l[universe_id]

        # get the hash value of the vector for this set of planes
        hash_value = hash_value_of_vector(v, planes)

        # get the hash table for this particular universe_id
        hash_table = hash_tables[universe_id]

        # get the list of document vectors for this hash table, where the key is the hash_value
        document_vectors_l = hash_table[hash_value]

        # get the id_table for this particular universe_id
        id_table = id_tables[universe_id]

        # get the subset of documents to consider as nearest neighbors from this id_table dictionary
        new_ids_to_consider = id_table[hash_value]

        # remove the id of the document that we're searching
        if doc_id in new_ids_to_consider:
            new_ids_to_consider.remove(doc_id)
            print(f"removed doc_id {doc_id} of input vector from new_ids_to_search")

        # loop through the subset of document vectors to consider
        for i, new_id in enumerate(new_ids_to_consider):

            # if the document ID is not yet in the set ids_to_consider...
            if new_id not in ids_to_consider_set:
                # access document_vectors_l list at index i to get the embedding
                # then append it to the list of vectors to consider as possible nearest neighbors
                document_vector_at_i = document_vectors_l[i]
                

                # append the new_id (the index for the document) to the list of ids to consider
                vecs_to_consider_l.append(document_vector_at_i)
                ids_to_consider_l.append(new_id)
                # also add the new_id to the set of ids to consider
                # (use this to check if new_id is not already in the IDs to consider)
                ids_to_consider_set.add(new_id)
       

    # Now run k-NN on the smaller set of vecs-to-consider.
    print("Fast considering %d vecs" % len(vecs_to_consider_l))

    # convert the vecs to consider set to a list, then to a numpy array
    vecs_to_consider_arr = np.array(vecs_to_consider_l)

    # call nearest neighbors on the reduced list of candidate vectors
    nearest_neighbor_idx_l = nearest_neighbor(v, vecs_to_consider_arr, k=k)

    # Use the nearest neighbor index list as indices into the ids to consider
    # create a list of nearest neighbors by the document ids
    nearest_neighbor_ids = [ids_to_consider_l[idx]
                            for idx in nearest_neighbor_idx_l]

    return nearest_neighbor_ids

In [129]:
#document_vecs, ind2Tweet
doc_id = 0
doc_to_search = all_tweets[doc_id]
vec_to_search = document_vecs[doc_id]

In [135]:
nearest_neighbor_ids = aprx_knn(doc_id, vec_to_search, planes_l, k=3, num_universes_to_use=5)

removed doc_id 0 of input vector from new_ids_to_search
removed doc_id 0 of input vector from new_ids_to_search
removed doc_id 0 of input vector from new_ids_to_search
removed doc_id 0 of input vector from new_ids_to_search
removed doc_id 0 of input vector from new_ids_to_search
Fast considering 1355 vecs


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


In [136]:
print(f"Nearest neighbors for document {doc_id}")
print(f"Document contents: {doc_to_search}")
print("")

for neighbor_id in nearest_neighbor_ids:
    print(f"Nearest neighbor at document id {neighbor_id}")
    print(f"document contents: {all_tweets[neighbor_id]}")

Nearest neighbors for document 0
Document contents: hopeless for tmr :(

Nearest neighbor at document id 2990
document contents: I'm starving :(
Nearest neighbor at document id 3030
document contents: 11:11 a boyfriend :(
Nearest neighbor at document id 5727
document contents: kill :) me :) http://t.co/5kon9Txmf6
